In [6]:
import pandas as pd
from datetime import time
from datetime import datetime, timedelta
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'seaborn'

In [7]:
# Строим график
def visual(df_prepare):
    df = df_prepare.copy()
    df['dif'] = df['rashod_cold'] - df['rashod_hot'] - 0.05
    plt.figure(figsize=(12, 4))
    fig, axes = plt.subplots(3, 1, figsize=(12, 8))

    sns.lineplot(data=df, x='datetime', y='rashod_hot', label='Расход горячей, м3/ч', linewidth=1, ax=axes[0])
    sns.lineplot(data=df, x='datetime', y='rashod_cold', label='Расход холодной, м3/ч', linewidth=1, ax=axes[0])
    sns.lineplot(data=df, x='datetime', y='dif', label='Утечка, м3/ч', linewidth=1, ax=axes[0])
    axes[0].set_xlabel('дата')
    axes[0].set_ylabel('Потребеление водыб м3/ч')

    sns.lineplot(data=df, x='datetime', y='input_hot', label='Подача горячей, м3/ч', ax=axes[1])
    sns.lineplot(data=df, x='datetime', y='output_hot', label='Выход горячей, м3/ч', linewidth=2, ax=axes[1])
    axes[1].set_xlabel('дата')
    axes[1].set_ylabel('Потребеление водыб м3/ч')

    sns.lineplot(data=df, x='datetime', y='temp_input', label='Температура входа', linewidth=1, ax=axes[2])
    sns.lineplot(data=df, x='datetime', y='temp_output', label='Температура выхода', linewidth=1, ax=axes[2])
    axes[2].set_xlabel('дата')
    axes[2].set_ylabel('T воды, град')

    plt.grid(True)
    plt.tight_layout()
    plt.show()


def generate_data(df_origin, first_day='2025-01-01', last_day='2025-12-07', id=0):
    df = df_origin.copy()
    df['day'] = df['datetime'].dt.day
    df['dayofweek'] = df['datetime'].dt.dayofweek
    df['dayofweek'] = df['dayofweek'].apply(lambda x: 0 if x <= 3 else 1)

    start_date = datetime.strptime(first_day, '%Y-%m-%d')
    end_date = datetime.strptime(last_day, '%Y-%m-%d')
    
    date_list = []
    current_date = start_date
    
    while current_date <= end_date:
        date_list.append(current_date)
        current_date += timedelta(days=1)

    temp_list = []
    for day in date_list:
        weekend = 1 if day.weekday() >= 4 else 0
        temp = df[df['dayofweek'] == weekend]

        random_day = random.choice(temp['day'].unique())
        temp = temp[temp['day'] == random_day]
        temp['time'] = temp['datetime'].dt.time
        temp['datetime'] = temp['time'].apply(lambda t: datetime.combine(day.date(), t))
       
        
        temp = temp[['datetime', 'input_hot', 'output_hot', 'rashod_hot', 'temp_input', 'temp_output', 'cumulative_rashod_cold', 'rashod_cold']]
        temp_list.append(temp)

    total_df = pd.concat(temp_list, axis=0)
    total_df.insert(0, 'id', id)



    total_df = total_df.reset_index(drop=True)

    my_list = [
        random.uniform(0.01,0.1),
        random.uniform(0.2,0.3),
        random.uniform(0.4,0.5),
        random.uniform(0.6,0.7),
        ]
    random_count = np.random.randint(2, 5)
    random_elements = np.random.choice(my_list, size=random_count, replace=False)

    for i in random_elements:

        # Определяем диапазон строк
        decrease_start = int(len(total_df) * i)
        decrease_end = decrease_start + random.randint(100, 270)
        zero_start = decrease_end +1
        zero_end = zero_start + random.randint(10,100)

        num_decrease_rows = decrease_end - decrease_start + 1
        num_zero_rows = zero_end - zero_start + 1

        # 1. Постепенное уменьшение с 300 по 564 строку
        for col in ['rashod_hot', 'input_hot', 'output_hot']:
            original_values = total_df.loc[decrease_start:decrease_end, col].values
            decrease_factors = np.linspace(1, 0, num_decrease_rows)
            total_df.loc[decrease_start:decrease_end, col] = original_values * decrease_factors

        # 2. Устанавливаем нули с 565 по 664 строку
        total_df.loc[zero_start:zero_end, ['rashod_hot', 'input_hot', 'output_hot']] = 0

        # Для temp_input - уменьшаем существующие значения на 10 градусов
        original_temp_input = total_df.loc[decrease_start:zero_end, 'temp_input'].values
        decrease_temp_input = np.linspace(0, 10, num_decrease_rows + num_zero_rows)
        total_df.loc[decrease_start:zero_end, 'temp_input'] = original_temp_input - decrease_temp_input

        # Для temp_output - уменьшаем существующие значения на 14 градусов
        original_temp_output = total_df.loc[decrease_start:zero_end, 'temp_output'].values
        decrease_temp_output = np.linspace(0, 14, num_decrease_rows + num_zero_rows)
        total_df.loc[decrease_start:zero_end, 'temp_output'] = original_temp_output - decrease_temp_output

    random_multiplier = random.uniform(1,10)
    total_df['input_hot'] = (total_df['input_hot'] * random_multiplier).round(2)
    total_df['output_hot'] = (total_df['output_hot'] * random_multiplier).round(2)
    total_df['rashod_hot'] = (total_df['rashod_hot'] * random_multiplier).round(2)
    total_df['rashod_cold'] = total_df['rashod_cold'] * random_multiplier
    total_df['cumulative_rashod_cold'] = total_df['rashod_cold'].cumsum().shift(1, fill_value=0) + np.random.uniform(5678, 15046)
    total_df['rashod_cold'] = total_df['rashod_cold'].round(2)
    total_df['cumulative_rashod_cold'] = total_df['cumulative_rashod_cold'].round(3)
    total_df['temp_output'] = total_df['temp_output'].astype(int)
    total_df['temp_input'] = total_df['temp_input'].astype(int)
    
    return total_df

In [9]:
df_hot= pd.read_excel('GVS.xlsx')
df_hot['Дата'] = pd.to_datetime(df_hot['Дата'], format='%d.%m.%Y', errors='coerce')
df_hot['datetime'] = df_hot['Дата'] + pd.to_timedelta(df_hot['Время суток, ч'].str.split('-').str[0].astype(int), unit='h')
df_hot.drop(['Дата', 'Время суток, ч'], axis=1, inplace=True)
df_hot.rename(columns={
    'Подача, м3': 'input_hot',
    'Обратка, м3': 'output_hot',
    'Потребление за период, м3': 'rashod_hot',
    'Т1 гвс, оС':'temp_input',
    'Т2 гвс, оС': 'temp_output'
    }, inplace=True)

df_hot


,input_hot,output_hot,rashod_hot,temp_input,temp_output,datetime
0,0.23,0.12,0.11,60,42,2025-04-01 00:00:00
1,0.09,0.05,0.04,60,42,2025-04-01 01:00:00
2,0.11,0.06,0.05,60,42,2025-04-01 02:00:00
3,0.11,0.06,0.05,60,42,2025-04-01 03:00:00
4,0.14,0.08,0.06,60,42,2025-04-01 04:00:00
...,...,...,...,...,...,...
715,0.00,0.00,0.00,48,27,2025-04-30 19:00:00
716,0.00,0.00,0.00,48,27,2025-04-30 20:00:00
717,0.00,0.00,0.00,48,27,2025-04-30 21:00:00
718,0.00,0.00,0.00,48,27,2025-04-30 22:00:00


In [10]:
df_cold= pd.read_excel('HVS.xlsx')
df_cold['Дата'] = pd.to_datetime(df_cold['Дата'], format='%d.%m.%Y', errors='coerce')
df_cold['datetime'] = df_cold['Дата'] + pd.to_timedelta(df_cold['Время суток, ч'].str.split('-').str[0].astype(int), unit='h')
df_cold.drop(['Дата', 'Время суток, ч'], axis=1, inplace=True)
df_cold.rename(columns={
    'Потребление накопленным итогом, м3': 'cumulative_rashod_cold',
    'Потребление за период, м3': 'rashod_cold'
    }, inplace=True)
df_cold

,cumulative_rashod_cold,rashod_cold,datetime
0,12345.600,0.11,2025-04-01 00:00:00
1,12345.706,0.04,2025-04-01 01:00:00
2,12345.749,0.05,2025-04-01 02:00:00
3,12345.800,0.05,2025-04-01 03:00:00
4,12345.851,0.06,2025-04-01 04:00:00
...,...,...,...
715,12470.775,0.39,2025-04-30 19:00:00
716,12471.169,0.33,2025-04-30 20:00:00
717,12471.501,0.25,2025-04-30 21:00:00
718,12471.750,0.21,2025-04-30 22:00:00


In [11]:
df = df_hot.merge(df_cold, on='datetime')
df


,input_hot,output_hot,rashod_hot,temp_input,temp_output,datetime,cumulative_rashod_cold,rashod_cold
0,0.23,0.12,0.11,60,42,2025-04-01 00:00:00,12345.600,0.11
1,0.09,0.05,0.04,60,42,2025-04-01 01:00:00,12345.706,0.04
2,0.11,0.06,0.05,60,42,2025-04-01 02:00:00,12345.749,0.05
3,0.11,0.06,0.05,60,42,2025-04-01 03:00:00,12345.800,0.05
4,0.14,0.08,0.06,60,42,2025-04-01 04:00:00,12345.851,0.06
...,...,...,...,...,...,...,...,...
715,0.00,0.00,0.00,48,27,2025-04-30 19:00:00,12470.775,0.39
716,0.00,0.00,0.00,48,27,2025-04-30 20:00:00,12471.169,0.33
717,0.00,0.00,0.00,48,27,2025-04-30 21:00:00,12471.501,0.25
718,0.00,0.00,0.00,48,27,2025-04-30 22:00:00,12471.750,0.21


In [12]:

temp = generate_data(df[df['datetime'] < datetime(2025, 4, 17)], first_day='2025-01-01', last_day='2025-03-31', id=1)

visual(temp)

NameError: name 'plt' is not defined

In [11]:
from tqdm import tqdm

list_pandas = []
for i in tqdm(range(1, 3)):
    temp = generate_data(df[df['datetime'] < datetime(2025, 4, 17)], first_day='2024-09-01', last_day='2025-09-20', id=i)
    list_pandas.append(temp)
    # temp.to_csv(f"{i}.csv", index=False)

total_df = pd.concat(list_pandas, axis=0)
total_df


100%|██████████| 2/2 [00:01<00:00,  1.04it/s]


,id,datetime,input_hot,output_hot,rashod_hot,temp_input,temp_output,cumulative_rashod_cold,rashod_cold
0,1,2024-09-01 00:00:00,2.40,1.48,0.92,58,39,8045.891,1.02
1,1,2024-09-01 01:00:00,0.92,0.55,0.37,58,39,8046.908,0.46
2,1,2024-09-01 02:00:00,1.11,0.65,0.46,58,39,8047.371,0.46
3,1,2024-09-01 03:00:00,1.11,0.65,0.46,58,39,8047.833,0.46
4,1,2024-09-01 04:00:00,1.48,0.92,0.55,58,39,8048.296,0.65
...,...,...,...,...,...,...,...,...,...
9235,2,2025-09-20 19:00:00,4.37,2.64,1.73,58,39,16478.295,1.83
9236,2,2025-09-20 20:00:00,3.22,1.92,1.30,58,39,16480.122,1.35
9237,2,2025-09-20 21:00:00,2.74,1.63,1.11,58,39,16481.468,1.15
9238,2,2025-09-20 22:00:00,2.31,1.39,0.91,58,39,16482.622,0.96


In [13]:
import clickhouse_connect
from datetime import datetime

# Создание клиента
client = clickhouse_connect.get_client(
    host='localhost',      # или IP сервера
    port=8123,            # HTTP порт (или 9000 для native)
    username='admin',
    password='password',
)

In [16]:
# Создание базы данных
client.command('CREATE DATABASE IF NOT EXISTS msk_database')

In [20]:
create_table_query = '''
CREATE TABLE IF NOT EXISTS msk_database.analytic (
    id Int32,
    datetime DateTime,
    input_hot Float32,
    output_hot Float32,
    rashod_hot Float32,
    temp_input Int32,
    temp_output Int32,
    cumulative_rashod_cold Float32,
    rashod_cold Float32
) ENGINE = MergeTree()
ORDER BY (datetime)
'''

client.command('DROP TABLE IF EXISTS msk_database.analytic')
client.command(create_table_query)
print("Таблица создана успешно!")

Таблица создана успешно!


In [21]:
client.insert_df('msk_database.analytic', total_df)

In [24]:
click_df = client.query_df(
    f"""
    SELECT *
    FROM msk_database.analytic
    ORDER BY datetime DESC
    LIMIT {total_df['id'].nunique() * 24 * 7}
    """)

click_df

,id,datetime,input_hot,output_hot,rashod_hot,temp_input,temp_output,cumulative_rashod_cold,rashod_cold
0,2,2025-09-20 23:00:00,1.39,0.87,0.53,58,39,16483.582031,0.58
1,1,2025-09-20 23:00:00,2.40,1.39,1.02,58,40,23231.785156,1.11
2,2,2025-09-20 22:00:00,2.31,1.39,0.91,58,39,16482.621094,0.96
3,1,2025-09-20 22:00:00,4.07,2.40,1.66,58,40,23229.935547,1.85
4,2,2025-09-20 21:00:00,2.74,1.63,1.11,58,39,16481.468750,1.15
...,...,...,...,...,...,...,...,...,...
331,1,2025-09-14 02:00:00,1.11,0.65,0.46,59,42,22959.382812,0.46
332,2,2025-09-14 01:00:00,0.48,0.29,0.19,58,39,16341.916992,0.19
333,1,2025-09-14 01:00:00,0.92,0.55,0.37,59,42,22959.013672,0.37
334,2,2025-09-14 00:00:00,1.15,0.67,0.48,58,39,16341.435547,0.48


In [26]:
from datetime import datetime
import time
# Текущее время округленное до часов вниз
current_hour = pd.Timestamp.now().floor('H')
print(f"Текущий час (округленный вниз): {current_hour}")

Текущий час (округленный вниз): 2025-09-25 10:00:00


In [40]:
while True:
    # Текущее время округленное до часов вниз
    current_hour = pd.Timestamp.now().floor('H')

    click_df = client.query_df(
        f"""
        SELECT *
        FROM msk_database.analytic
        ORDER BY datetime DESC
        LIMIT {100 * 24 * 7}
        """)


    if click_df['datetime'].max() == current_hour:
        print(f"Максимальное время в таблице {click_df['datetime'].max()}")
        time.sleep(10)
        continue
    else:
        list_df = []
        for id in click_df['id'].unique():
            temp = click_df[(click_df['id'] == id) & (click_df['datetime'] == click_df['datetime'].max() - pd.Timedelta(days=6, hours=23))]
            list_df.append(temp)
        
        new_df = pd.concat(list_df, axis=0)
        new_df['datetime'] = new_df['datetime'] + pd.Timedelta(weeks=1)
        new_df['cumulative_rashod_cold'] = new_df['cumulative_rashod_cold'] + new_df['rashod_cold']
        new_df = new_df.reset_index(drop=True)
        client.insert_df('msk_database.analytic', new_df)
        time.sleep(10)
        print(f"Максимальное время в НОВОЙ таблице {new_df['datetime'].max()}")

            

Максимальное время в НОВОЙ таблице 2025-09-25 11:00:00
Максимальное время в таблице 2025-09-25 11:00:00
Максимальное время в таблице 2025-09-25 11:00:00


KeyboardInterrupt: 

## Predictive

In [14]:
import pandas as pd
from datetime import timedelta

def add_hourly_rows(df, hours_to_add=10):
    """
    Добавляет указанное количество строк для каждого уникального id
    с увеличением datetime на 1 час для каждой новой строки
    """
    # Находим максимальную дату для каждого id
    max_dates = df.groupby('id')['datetime'].max().reset_index()
    max_dates.rename(columns={'datetime': 'max_datetime'}, inplace=True)
    
    # Создаем список для новых строк
    new_rows = []
    
    # Для каждого уникального id
    for id_val in df['id'].unique():
        # Находим максимальную дату для этого id
        max_date = max_dates[max_dates['id'] == id_val]['max_datetime'].iloc[0]
        
        # Берем последнюю строку для этого id как шаблон
        last_row = df[df['id'] == id_val].iloc[-1].copy()
        
        # Создаем 10 новых строк
        for i in range(1, hours_to_add + 1):
            new_row = last_row.copy()
            new_row['datetime'] = max_date + timedelta(hours=i)
            # Обнуляем или оставляем пустыми некоторые поля (по желанию)
            new_row['input_hot'] = None  # или None, или оставить как в последней строке
            new_row['output_hot'] = None
            new_row['rashod_hot'] = None
            new_row['rashod_cold'] = None
            new_row['dif'] = None
            new_row['percent_dif'] = None
            new_row['temp_input'] = None
            new_row['temp_output'] = None
            # cumulative_rashod_cold можно оставить как в последней строке или увеличить
            
            new_rows.append(new_row)
    
    # Создаем DataFrame из новых строк
    new_df = pd.DataFrame(new_rows)
    
    # Объединяем с исходным DataFrame
    result_df = pd.concat([df, new_df], ignore_index=True)
    
    # Сортируем по id и datetime
    result_df = result_df.sort_values(['id', 'datetime']).reset_index(drop=True)
    
    return result_df

In [15]:
import clickhouse_connect
from datetime import datetime



In [157]:
def prepare_data_for_predict():
    # Создание клиента
    client = clickhouse_connect.get_client(
        host='localhost',      # или IP сервера
        port=8123,            # HTTP порт (или 9000 для native)
        username='admin',
        password='password',
    )

    df = client.query_df(
        f"""
        SELECT *
        FROM msk_database.analytic
        ORDER BY id, datetime
        """
        )

    df['dif'] = df['rashod_cold'] - df['rashod_hot']
    df['percent_dif'] =  df['dif'] / df['rashod_cold'] * 100 #Будет нашим таргетом
    del df['cumulative_rashod_cold']


    # # Сдвигаем таргет для трех моделей и добавляем лаговые признаки
    # df['target_4h'] = df.groupby('id')['percent_dif'].shift(-4)
    # df['target_24h'] = df.groupby('id')['percent_dif'].shift(-24)
    # df['target_72h'] = df.groupby('id')['percent_dif'].shift(-72)
    for i in tqdm(range(1, 30)):

        df[f'percent_lag{i}'] = df.groupby('id')['percent_dif'].shift(i)
        df[f'dif_lag{i}'] = df.groupby('id')['dif'].shift(i)
        df[f'temp_output_lag{i}'] = df.groupby('id')['temp_output'].shift(i)
    
    return df


def load_model():
    print('Загрузка моделей...')
    model_4 = CatBoostRegressor()
    model_4.load_model('model_4h.cbm')

    model_24 = CatBoostRegressor()
    model_24.load_model('model_24h.cbm')

    model_72 = CatBoostRegressor()
    model_72.load_model('model_72h.cbm')

    return model_4, model_24, model_72

def get_predict(df, model_4, model_24, model_72):
    df_clear = df.dropna()

    pred_4 = model_4.predict(df_clear.drop(columns=['datetime']))
    pred_24 = model_24.predict(df_clear.drop(columns=['datetime']))
    pred_72 = model_72.predict(df_clear.drop(columns=['datetime']))

    df_clear = df_clear[['id', 'datetime', 'dif', 'percent_dif']]
    df_clear['pred_4h'] = pred_4
    df_clear['pred_24h'] = pred_24
    df_clear['pred_72h'] = pred_72

    # Находим максимальную дату для каждого id
    max_dates = df_clear.groupby('id')['datetime'].max().reset_index()
    max_dates.rename(columns={'datetime': 'max_datetime'}, inplace=True)
    
    # Создаем список для новых строк
    new_rows = []
    
    # Для каждого уникального id
    for id_val in df_clear['id'].unique():
        # Находим максимальную дату для этого id
        max_date = max_dates[max_dates['id'] == id_val]['max_datetime'].iloc[0]
        
        # Берем последнюю строку для этого id как шаблон
        last_row = df_clear[df_clear['id'] == id_val].iloc[-1].copy()
        
        # Создаем 72 новых строк
        for i in range(1, 73):
            new_row = last_row.copy()
            new_row['datetime'] = max_date + timedelta(hours=i)
            # Обнуляем или оставляем пустыми некоторые поля (по желанию)
            new_row['dif'] = np.nan
            new_row['percent_dif'] = np.nan
            new_row['pred_4h'] = np.nan
            new_row['pred_24h'] = np.nan
            new_row['pred_72h'] = np.nan
            
            new_rows.append(new_row)
    
    # Создаем DataFrame из новых строк
    new_df = pd.DataFrame(new_rows)
    
    # Объединяем с исходным DataFrame
    result_df = pd.concat([df_clear, new_df], ignore_index=True)
    
    # Сортируем по id и datetime
    result_df = result_df.sort_values(['id', 'datetime']).reset_index(drop=True)

    result_df['pred_4h'] = result_df.groupby('id')['pred_4h'].shift(4)
    result_df['pred_24h'] = result_df.groupby('id')['pred_24h'].shift(24)
    result_df['pred_72h'] = result_df.groupby('id')['pred_72h'].shift(72)


    return result_df

In [158]:
df = prepare_data_for_predict()

model_4, model_24, model_72 = load_model()

100%|██████████| 29/29 [00:02<00:00, 14.50it/s]


Загрузка моделей...


In [167]:



total = get_predict(df, model_4, model_24, model_72)

total

,id,datetime,dif,percent_dif,pred_4h,pred_24h,pred_72h
0,1,2024-10-02 05:00:00,0.14,15.730336,NaN,NaN,NaN
1,1,2024-10-02 06:00:00,0.04,7.547163,NaN,NaN,NaN
2,1,2024-10-02 07:00:00,0.04,7.547163,NaN,NaN,NaN
3,1,2024-10-02 08:00:00,0.13,13.978495,NaN,NaN,NaN
4,1,2024-10-02 09:00:00,0.13,17.333334,13.956551,NaN,NaN
...,...,...,...,...,...,...,...
871695,100,2025-09-30 05:00:00,NaN,NaN,NaN,NaN,8.390816
871696,100,2025-09-30 06:00:00,NaN,NaN,NaN,NaN,7.944326
871697,100,2025-09-30 07:00:00,NaN,NaN,NaN,NaN,8.264752
871698,100,2025-09-30 08:00:00,NaN,NaN,NaN,NaN,7.568568


In [170]:
from sklearn.metrics import mean_absolute_error
mae_4h = total[['percent_dif', 'pred_4h']].dropna()
mae_4h = mean_absolute_error(mae_4h['percent_dif'], mae_4h['pred_4h'])
mae_4h

2.7632709218314733

In [134]:
from sklearn.model_selection import train_test_split

# Разделение на тренировочную и тестовую выборки
train_df, test_df = train_test_split(temp.sort_values(['datetime']), test_size=0.3, random_state=42, shuffle=False)


In [137]:
X_train = train_df.drop(columns=['datetime', 'target_4h', 'target_24h', 'target_72h'])
y_train = train_df['target_4h']
X_test = test_df.drop(columns=['datetime', 'target_4h', 'target_24h', 'target_72h'])
y_test = test_df['target_4h']



In [139]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
# Создание модели с базовыми параметрами
model_4 = CatBoostRegressor(iterations=200, learning_rate=0.1, depth=6, verbose=50)

# Обучение модели
model_4.fit(X_train, y_train, eval_set=(X_test, y_test), cat_features=['id'])

model_4.save_model('model_4h.cbm')
model = CatBoostRegressor()
model.load_model('model_4h.cbm')
# Оценка качества (например, RMSE)
rmse = mean_squared_error(y_test, model.predict(X_test))
print(f"RMSE тест: {rmse:.4f}")

# Оценка качества (например, RMSE)
rmse = mean_squared_error(y_train, model.predict(X_train))
print(f"RMSE train: {rmse:.4f}")

0:	learn: 19.2760777	test: 11.0747680	best: 11.0747680 (0)	total: 173ms	remaining: 34.3s
50:	learn: 5.8051708	test: 5.0607815	best: 5.0607815 (50)	total: 7.83s	remaining: 22.9s
100:	learn: 5.5257336	test: 4.7962203	best: 4.7962203 (100)	total: 15.4s	remaining: 15.1s
150:	learn: 5.3733019	test: 4.6654544	best: 4.6654544 (150)	total: 23.1s	remaining: 7.49s
199:	learn: 5.2509915	test: 4.5963386	best: 4.5963386 (199)	total: 30.5s	remaining: 0us

bestTest = 4.596338643
bestIteration = 199

RMSE тест: 21.1263
RMSE train: 27.4830


In [143]:
X_train = train_df.drop(columns=['datetime', 'target_4h', 'target_24h', 'target_72h'])
y_train = train_df['target_24h']
X_test = test_df.drop(columns=['datetime', 'target_4h', 'target_24h', 'target_72h'])
y_test = test_df['target_24h']

In [144]:

# Создание модели с базовыми параметрами
model_24 = CatBoostRegressor(iterations=200, learning_rate=0.1, depth=6, verbose=50)

# Обучение модели
model_24.fit(X_train, y_train, eval_set=(X_test, y_test), cat_features=['id'])

model_24.save_model('model_24h.cbm')
model = CatBoostRegressor()
model.load_model('model_24h.cbm')
# Оценка качества (например, RMSE)
rmse = mean_squared_error(y_test, model.predict(X_test))
print(f"RMSE тест: {rmse:.4f}")

# Оценка качества (например, RMSE)
rmse = mean_squared_error(y_train, model.predict(X_train))
print(f"RMSE train: {rmse:.4f}")

0:	learn: 19.4693646	test: 11.0358684	best: 11.0358684 (0)	total: 180ms	remaining: 35.8s
50:	learn: 8.3431234	test: 6.8856486	best: 6.8856486 (50)	total: 7.38s	remaining: 21.5s
100:	learn: 8.0896572	test: 6.8426784	best: 6.8426784 (100)	total: 14.5s	remaining: 14.2s
150:	learn: 7.9493646	test: 6.8243816	best: 6.8234131 (145)	total: 21.9s	remaining: 7.1s
199:	learn: 7.8280388	test: 6.8223508	best: 6.8206902 (192)	total: 28.9s	remaining: 0us

bestTest = 6.820690196
bestIteration = 192

Shrink model to first 193 iterations.
RMSE тест: 46.5218
RMSE train: 60.8534


In [141]:
X_train = train_df.drop(columns=['datetime', 'target_4h', 'target_24h', 'target_72h'])
y_train = train_df['target_72h']
X_test = test_df.drop(columns=['datetime', 'target_4h', 'target_24h', 'target_72h'])
y_test = test_df['target_72h']

In [142]:
model_72 = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, verbose=50)

# Обучение модели
model_72.fit(X_train, y_train, eval_set=(X_test, y_test), cat_features=['id'])

model_72.save_model('model_72h.cbm')
model = CatBoostRegressor()
model.load_model('model_72h.cbm')

# Оценка качества (например, RMSE)
rmse = mean_squared_error(y_test, model.predict(X_test))
print(f"RMSE тест: {rmse:.4f}")

# Оценка качества (например, RMSE)
rmse = mean_squared_error(y_train, model.predict(X_train))
print(f"RMSE train: {rmse:.4f}")

0:	learn: 19.7893966	test: 10.6703946	best: 10.6703946 (0)	total: 183ms	remaining: 3m 3s
50:	learn: 10.4026002	test: 7.0540286	best: 7.0540286 (50)	total: 7.22s	remaining: 2m 14s
100:	learn: 10.0608386	test: 6.9876485	best: 6.9876485 (100)	total: 14.6s	remaining: 2m 10s
150:	learn: 9.8122328	test: 6.9784730	best: 6.9779616 (142)	total: 21.9s	remaining: 2m 3s
200:	learn: 9.6565225	test: 6.9561800	best: 6.9561800 (200)	total: 29.4s	remaining: 1m 56s
250:	learn: 9.5410259	test: 6.9481707	best: 6.9481362 (249)	total: 36.8s	remaining: 1m 49s
300:	learn: 9.4366457	test: 6.9432244	best: 6.9425377 (296)	total: 44.2s	remaining: 1m 42s
350:	learn: 9.3689672	test: 6.9372625	best: 6.9372625 (350)	total: 51.5s	remaining: 1m 35s
400:	learn: 9.3008499	test: 6.9328660	best: 6.9326598 (399)	total: 58.7s	remaining: 1m 27s
450:	learn: 9.2480317	test: 6.9286172	best: 6.9286172 (450)	total: 1m 6s	remaining: 1m 20s
500:	learn: 9.1866440	test: 6.9243402	best: 6.9243402 (500)	total: 1m 13s	remaining: 1m 13s
5

In [131]:
test_df

,id,datetime,input_hot,output_hot,rashod_hot,temp_input,temp_output,rashod_cold,dif,percent_dif,...,temp_output_lag44,percent_lag45,dif_lag45,temp_output_lag45,percent_lag46,dif_lag46,temp_output_lag46,percent_lag47,dif_lag47,temp_output_lag47
551414,64,2025-06-08 23:00:00,2.29,1.34,0.95,59,42,0.95,0.00,0.000000,...,41.0,0.000000,0.00,41.0,20.000004,0.08,41.0,0.000000,0.00,41.0
231105,27,2025-06-08 23:00:00,1.42,0.82,0.60,58,40,0.66,0.06,9.090909,...,41.0,0.000000,0.00,41.0,18.518522,0.05,41.0,0.000000,0.00,41.0
689926,80,2025-06-08 23:00:00,1.96,1.20,0.76,57,38,0.91,0.15,16.483521,...,41.0,0.000000,0.00,41.0,21.212128,0.07,41.0,0.000000,0.00,41.0
473501,55,2025-06-08 23:00:00,0.98,0.58,0.41,59,42,0.41,0.00,0.000000,...,40.0,17.647058,0.03,40.0,0.000000,0.00,40.0,8.823529,0.03,40.0
438873,51,2025-06-08 23:00:00,1.45,0.89,0.56,54,33,1.13,0.57,50.442474,...,37.0,35.000004,0.14,37.0,18.750002,0.06,37.0,25.925922,0.21,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605917,70,2025-09-23 16:00:00,3.70,2.59,1.11,59,41,1.29,0.18,13.953485,...,40.0,10.249305,0.37,40.0,7.335900,0.19,40.0,8.866993,0.18,40.0
614574,71,2025-09-23 16:00:00,2.62,1.80,0.82,59,40,0.90,0.08,8.888887,...,40.0,15.410959,0.45,40.0,10.952376,0.23,40.0,13.939396,0.23,40.0
623231,72,2025-09-23 16:00:00,3.56,2.49,1.07,59,41,1.25,0.18,14.399997,...,40.0,15.273773,0.53,40.0,10.843372,0.27,40.0,13.775510,0.27,40.0
640545,74,2025-09-23 16:00:00,2.53,1.75,0.78,60,42,0.78,0.00,0.000000,...,41.0,6.949800,0.18,41.0,6.217617,0.12,41.0,8.609271,0.13,41.0


In [130]:
len(y_pred)

171640